In [114]:
import pandas as pd
import json

In [115]:
with open('chessbuds_messages.json') as j:
    data = json.load(j)

In [116]:
type(data)

dict

In [117]:
data.keys()

dict_keys(['participants', 'messages', 'title', 'is_still_participant', 'thread_type', 'thread_path', 'magic_words', 'joinable_mode'])

In [118]:
type(data['participants'])

list

In [119]:
type(data['messages'])

list

In [120]:
len(data['messages'])

223

In [121]:
data['messages'][0].keys()

dict_keys(['sender_name', 'timestamp_ms', 'content', 'reactions', 'type', 'is_unsent', 'is_taken_down', 'bumped_message_metadata'])

In [122]:
chessDf = pd.DataFrame(data['messages'])

In [123]:
chessDf.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users
0,Joanna Rusch,1666374933946,Maybe he just wants to ride the publicity for ...,"[{'reaction': 'ð', 'actor': 'Chad Larson'},...",Generic,False,False,{'bumped_message': 'Maybe he just wants to rid...,NaN,NaN,NaN,NaN
1,Chad Larson,1666373448613,To be fair to Hans....no one wants to be assoc...,"[{'reaction': 'ð', 'actor': 'Scott Pence'},...",Generic,False,False,{'bumped_message': 'To be fair to Hans....no o...,NaN,NaN,NaN,NaN
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and tha...,"[{'reaction': 'ð', 'actor': 'Scott Pence'},...",Generic,False,False,{'bumped_message': 'He would have to prove he ...,NaN,NaN,NaN,NaN
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a...","[{'reaction': 'ð', 'actor': 'Chad Larson'},...",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over sho...",NaN,NaN,NaN,NaN
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. ...",NaN,Generic,False,False,"{'bumped_message': 'From what I see, I don't t...",NaN,NaN,NaN,NaN


In [124]:
type(chessDf['reactions'].iloc[0])

list

In [125]:
type(chessDf['reactions'].iloc[0][0])

dict

In [126]:
chessDf['reactions'].iloc[0][0]['reaction']

'ð\x9f\x91\x8d'

In [157]:
chessDf['reactions'] = chessDf['reactions'].fillna('')

In [163]:
chessDf['share'] = chessDf['share'].fillna('')

In [164]:
chessDf['photos'] = chessDf['photos'].fillna('')

In [165]:
chessDf['gifs'] = chessDf['gifs'].fillna('')

In [166]:
chessDf['users'] = chessDf['users'].fillna('')

In [167]:
chessDf['reactions'] = [[y['reaction'] for y in x] for x in chessDf['reactions']]

TypeError: string indices must be integers, not 'str'

In [168]:
chessDf.head()

,sender_name,timestamp_ms,content,reactions,type,is_unsent,is_taken_down,bumped_message_metadata,share,photos,gifs,users
0,Joanna Rusch,1666374933946,Maybe he just wants to ride the publicity for ...,"[ð, ð]",Generic,False,False,{'bumped_message': 'Maybe he just wants to rid...,,,,
1,Chad Larson,1666373448613,To be fair to Hans....no one wants to be assoc...,"[ð, ð]",Generic,False,False,{'bumped_message': 'To be fair to Hans....no o...,,,,
2,Chad Larson,1666373216381,He would have to prove he didn't cheat and tha...,"[ð, ð]",Generic,False,False,{'bumped_message': 'He would have to prove he ...,,,,
3,Scott Pence,1666373164883,"Yeah, no way. You over shoot and hope to get a...","[ð, ð]",Generic,False,False,"{'bumped_message': 'Yeah, no way. You over sho...",,,,
4,Chad Larson,1666373111157,"From what I see, I don't think he could win. ...",[],Generic,False,False,"{'bumped_message': 'From what I see, I don't t...",,,,
